In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import roc_auc_score
from scipy.optimize import minimize, Bounds
from sklearn.model_selection import train_test_split

In [ ]:
# use cuda

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device {}".format(device))

In [ ]:
# load data

data = np.load("test_mc.npy")
theta0, theta1 = train_test_split(data, test_size=0.5, shuffle=True)

theta0 = torch.Tensor(theta0)
theta1 = torch.Tensor(theta1)

In [ ]:
# function to calculate weights analytically

def cal_weight(xx1, xx2, xx3, phi, costh):
    weight = 1. + xx1* costh* costh + 2.* xx2* costh* torch.sqrt(1. - costh* costh)* torch.cos(phi) + 0.5* xx3* (1. - costh* costh)* torch.cos(2.* phi)
    return weight/(1. + costh* costh)

In [ ]:
# define the neural network

class Net(nn.Module):
    def __init__(self, in_features=2, out_features=1):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(in_features, 32, bias=True)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 32, bias=True)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1, bias=True)
        self.act3 = nn.ReLU()
        
    def forward(self, x):
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))
        x = self.act3(self.fc3(x))
        return x

In [ ]:
net = Net().to(device)
total_trainable_params = sum(p.numel() for p in net.parameters())
print('total trainable params:', total_trainable_params)

In [ ]:
# unknown parameters

LAMBDA_0, MU_0, NU_0 = 0.4, 0.0, 0.2 

In [ ]:
epochs = 15
batch_size = 1024

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
# run classifier
# true_{mass, pT, xF, phi, costh} and reco_{mass, pT, xF, phi, costh}

def run_classifier(x1, x2, x3, fit_type='generator'):
    
    if fit_type == 'generator':
        theta0_G = theta0[:, 3:5]
        weight0_G = cal_weight(LAMBDA_0, MU_0, NU_0, theta0[:, 3], theta0[:, 4]).reshape(-1, 1)
        label0_G = torch.zeros(theta0_G.shape[0], dtype=torch.float32).reshape(-1, 1)
        
        theta1_G = theta1[:, 3:5]
        weight1_G = cal_weight(x1, x2, x3, theta1[:, 3], theta1[:, 4]).reshape(-1, 1)
        label1_G = torch.ones(theta1_G.shape[0], dtype=torch.float32).reshape(-1, 1)
        
        theta = torch.cat((theta0_G, theta1_G), 0)
        weight = torch.cat((weight0_G, weight1_G), 0)
        label = torch.cat((label0_G, label1_G), 0)
        
        # print("min: {}, max: {}".format(torch.min(weight1_G), torch.max(weight1_G)))
        
    elif fit_type == 'detector':
        theta0_S = theta0[:, 8:]
        weight0_S = cal_weight(LAMBDA_0, MU_0, NU_0, theta0[:, 3], theta0[:, 4]).reshape(-1, 1)
        label0_S = torch.zeros(theta0_S.shape[0], dtype=torch.float32).reshape(-1, 1)
        
        theta1_S = theta1[:, 8:]
        weight1_S = cal_weight(x1, x2, x3, theta1[:, 3], theta1[:, 4]).reshape(-1, 1)
        label1_S = torch.ones(theta1_S.shape[0], dtype=torch.float32).reshape(-1, 1)
        
        theta = torch.cat((theta0_S, theta1_S), 0)
        weight = torch.cat((weight0_S, weight1_S), 0)
        label = torch.cat((label0_S, label1_S), 0)
        
        # print("min: {}, max: {}".format(torch.min(weight1_S), torch.max(weight1_S)))
    
    else:
        raise ValueError("fit type must be set to 'generator' or 'detector'")
        
    dataset = TensorDataset(theta.to(device), weight.to(device), label.to(device))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    loss = []
    for epoch in range(epochs):
        net.train()
        run_loss, m = 0., 0.
        for inputs, weights, targets in dataloader:
            optimizer.zero_grad()
            
            outputs = net(inputs)
            criterion.weight = weights
            loss_batch = criterion(outputs, targets)
            
            loss_batch.backward()
            optimizer.step()
            
            run_loss += loss_batch.item()
            m += 1.0
            
        loss.append(run_loss / m)
        # print("epoch : {} loss : {}".format(epoch + 1, run_loss / m))
        
    # plt.figure(figsize=(6, 5))
    # plt.plot(loss)
    # plt.xlabel("epoch")
    # plt.ylabel("loss")
    # # plt.savefig("../imgs/loss.png")
    # # plt.close("all")
    # plt.show()
    
    net.eval()
    outputs = net(theta.to(device)).cpu().detach().numpy()
    auc = roc_auc_score(label, outputs, sample_weight=weight)
    return auc

In [ ]:
# scan the minimum values

runs = np.linspace(-1., 1., 11)

AUC_G = []
AUC_S = []


def scan_theta():
    for run in runs:
        print("*** generator level ***")
        print("Testing theta = {:.1f}".format(run))
        
        auc = run_classifier(LAMBDA_0, MU_0, run, "generator")
        AUC_G.append(auc)
        
        print("AUC: {:.2f}".format(auc))
        print("\n")
        pass
    
        print("*** detector level ***")
        print("Testing theta = {:.1f}".format(run))
        auc = run_classifier(LAMBDA_0, MU_0, run, "detector")
        AUC_S.append(auc)
        
        print("AUC: {:.2f}".format(auc))
        print("\n")
        pass

In [ ]:
# use minimizer to extract parameters

def get_AUC_G(x):
    AUC_G = run_classifier(LAMBDA_0, MU_0, x[0], "generator")
    return AUC_G


def get_AUC_S(x):
    AUC_S = run_classifier(LAMBDA_0, MU_0, x[0], "detector")
    return AUC_S

In [ ]:
# restrict optimization to bounds of parameterization of the reweighting function
mybounds = Bounds(np.array([0.0]), np.array([0.5]))

THETA_G = []
FUNC_G = []
THETA_S = []
FUNC_S = []


def run_minimizer():
    for i in range(20):
        theta_prime = np.random.uniform(0.0, 0.5, 1)
        print("iteration : {} theta prime = {:.2f}".format(i+1, theta_prime[0]))
        res = minimize(get_AUC_G, theta_prime, method="Powell", bounds=mybounds, options={
                                                                            'maxiter': 50,
                                                                            'disp': True,
                                                                            'return_all': True
                                                                            })
        
        THETA_G.append(res["x"][0])
        FUNC_G.append(res["fun"])
        
        print("best theta : ({:.2f})".format(res["x"][0]))
        print("\n")
        
        res = minimize(get_AUC_S, theta_prime, method="Powell", bounds=mybounds, options={
                                                                            'maxiter': 50,
                                                                            'disp': True,
                                                                            'return_all': True
                                                                            })
        
        THETA_S.append(res["x"][0])
        FUNC_S.append(res["fun"])
        
        print("best theta : ({:.2f})".format(res["x"][0]))
        print("\n")

In [ ]:
# run functions

scan_theta()
run_minimizer()

In [ ]:
print("best fit generator level = {:.2f} +/- {:.2f}".format(np.mean(THETA_G), np.std(THETA_G)))
print("best fit detector level = {:.2f} +/- {:.2f}".format(np.mean(THETA_S), np.std(THETA_S)))

In [ ]:
plt.figure(figsize=(6, 5))
plt.plot(runs, AUC_G, label='Generator-level')
plt.plot(runs, AUC_S, label='Detector-level')
plt.errorbar(np.mean(THETA_G), 0.52, xerr=np.std(THETA_G), fmt="o", label=r'$\theta_G = {:.2f} \pm {:.2f}$'.format(np.mean(THETA_G), np.std(THETA_G)))
plt.errorbar(np.mean(THETA_S), 0.52, xerr=np.std(THETA_S), fmt="o", label=r'$\theta_S = {:.2f} \pm {:.2f}$'.format(np.mean(THETA_S), np.std(THETA_S)))
plt.xlabel(r'$\nu$')
plt.ylabel('AUC')
plt.legend(loc='lower left', frameon=False)
# plt.savefig("../imgs/NNFit1D.png")
# plt.close("close")
plt.show()